# Getting started with DeepMatcher

Note: you can run **[this notebook live in Google Colab](https://colab.research.google.com/github/sidharthms/deepmatcher/blob/master/examples/getting_started.ipynb)** and use free GPUs provided by Google.

This tutorial describes how to effortlessly perform entity matching using deep neural networks. Specifically, we will see how to match pairs of tuples (also called data records or table rows) to determine if they refer to the same real world entity. To do so, we will need labeled examples as input, i.e., tuple pairs which have been annotated as matches or non-matches. This will be used to train our neural network using supervised learning. At the end of this tutorial, you will have a trained neural network as output which you can easily apply to unlabeled tuple pairs to make predictions.

As an overview, here are the 4 steps to use `deepmatcher` which we will go through in this tutorial:

<ol start="0">
  <li>Setup</li>
  <li>Process data</li>
  <li>Define neural network model</li>
  <li>Train model</li>
  <li>Apply model to new data</li>
</ol>

Let's begin!

## Step 0. Setup

If you are running this notebook inside Colab, you will first need to install necessary packages by running the code below:

In [4]:
!pip install -q http://download.pytorch.org/whl/cu80/torch-0.3.1-cp36-cp36m-linux_x86_64.whl
!pip install -q --process-dependency-links git+https://github.com/sidharthms/deepmatcher

Now let's import `deepmatcher` which will do all the heavy lifting to build and train neural network models for entity matching. 

In [ ]:
import deepmatcher as dm

We recommend having a GPU available for the training in Step 4. In case a GPU is not available, we will use all available CPU cores. You can run the following command to determine if a GPU is available and will be used for training:

In [ ]:
import torch
torch.cuda.is_available()

### Download sample data for entity matching

Now let's get some sample data to play with in this tutorial. We will need three sets of labeled data and one set of unlabeled data:

1. **Training Data:** This is used for training our neural network model.
2. **Validation Data:** This is used for determining the configuration (i.e., hyperparameters) of our model in such a way that the model does not overfit to the training set.
3. **Test Data:** This is used to estimate the performance of our trained model on unlabeled data.
4. **Unlabeled Data:** The trained model is applied on this data to obtain predictions, which can then be used for downstream tasks in practical application scenarios.

We download these four data sets to the `sample_data` directory:

In [0]:
!mkdir -p sample_data
!wget -qnc -P sample_data https://raw.githubusercontent.com/sidharthms/deepmatcher/master/examples/sample_data/amz_goog_train.csv
!wget -qnc -P sample_data https://raw.githubusercontent.com/sidharthms/deepmatcher/master/examples/sample_data/amz_goog_validation.csv
!wget -qnc -P sample_data https://raw.githubusercontent.com/sidharthms/deepmatcher/master/examples/sample_data/amz_goog_test.csv
!wget -qnc -P sample_data https://raw.githubusercontent.com/sidharthms/deepmatcher/master/examples/sample_data/amz_goog_unlabeled.csv

## Step 1. Process data

Before we can use our data for training, `deepmatcher` needs to first load and process it in order to prepare it for neural network training. Currently `deepmatcher` only supports processing CSV files. Each CSV file is assumed to have the following kinds of columns:

* **"Left" attributes (required):** Our goal is to match tuple pairs. "Left" attributes are columns that correspond to the "left" tuple or the first tuple in the tuple pair. These column names are expected to be prefixed with "left_" by default.
* **"Right" attributes (required):** "Right" attributes are columns that correspond to the "right" tuple or the second tuple in the tuple pair. These column names are expected to be prefixed with "right_" by default.
* **Label column (required for train, validation, test):** Column containing the labels (match or non-match) for each tuple pair. Expected to be named "label" by default
* **ID column (required):** Column containing a unique ID for each tuple pair. This is for evaluation convenience.  Expected to be named "id" by default.

More details on what data processing involves and ways to customize it are described in **[this notebook](https://github.com/sidharthms/deepmatcher/tree/master/examples/data_process.ipynb)**. 

### Processing train / validation / test data
In order to process our train, validation and test CSV files we call `dm.process` in the following code snippet which will load and process the CSV files and return three processed `MatchingDataset` objects respectively. These dataset objects will later be used for training and evaluation. The basic parameters to `dm.proecss` are as follows:

* **path (required): ** The path where all data is stored. This includes train, validation and test. `deepmatcher` may create new files in this directory to store information about these data sets. This allows subsequent `dm.process` calls to be much faster.
* **train (required): ** File name of training data in `path` directory.
* **validation (required): ** File name of validation data in `path` directory.
* **test (optional): ** File name of test data in `path` directory.
* **ignore_columns (optional): ** Any columns in the CSV files that you may want to ignore for the purposes of training. These should be included here. 

Note that the train, validation and test CSVs must all share the same schema, i.e., they should have the same columns. Processing data involves several steps and can take several minutes to complete, especially if this is the first time you are running the `deepmatcher` package.

NOTE: If you are running this in Colab, you may get a message saying 'Memory usage is close to the limit.' You can safely ignore it for now. We are working on reducing the memory footprint.

In [3]:
train, validation, test = dm.process(
    path='sample_data',
    train='amz_goog_train.csv',
    validation='amz_goog_validation.csv',
    test='amz_goog_test.csv',
    ignore_columns=('left_id', 'right_id'))


Load time: 3.7080047791823745
Vocab time: 13.488945204764605
Metadata time: 1.5047426847741008
Cache time: 2.229101464152336


#### Peeking at processed data
Let's take a look at how the processed data looks like. To do this, we get the raw `pandas` table corresponding to the processed training dataset object. 

In [ ]:
train_table = train.get_raw_table()
train_table.head()

As you can see, the processed attribute values have been tokenized and lowercased so don't look exactly the same as the input training data. These modifications help the neural network generalize better, i.e., perform better on data not trained on. 

### Processing unlabeled data

`dm.process` can also be used to process unlabeled data, as shown in the code snippet below, so that you can perform prediction over it. The basic parameters to use `dm.process` for this case are as follows:

* **path (required): ** The path where unlabeled data is stored.
* **unlabeled (required): ** File name of unlabeled data in `path` directory.
* **ignore_columns (optional): ** Any columns in the CSV file that you may want to ignore for the purposes of training. These should be included here.

Note that the unlabeled CSV file must have the same schema as the train, validation and test CSVs.

In [ ]:
unlabeled = dm.process(
    path='sample_data',
    unlabeled='amz_goog_unlabeled.csv',
    ignore_columns=('left_id', 'right_id'))

## Step 2. Define neural network model

In this step you tell `deepmatcher` what kind of neural network you would like to use for entity matching. The easiest way to do this is to use one of the several kinds of neural network models that comes built-in with `deepmatcher`. To use a built-in network, construct a `dm.MatchingModel` as follows:

`model = dm.MatchingModel(attr_summarizer='<TYPE>')`

where `<TYPE>` is one of `sif`, `rnn`, `attention` or `hybrid`. If you are not familiar with what these mean, we strongly recommend taking a look at either **[slides from our talk on deepmatcher](http://bit.do/deepmatcher-talk)** for a high level overview, or **[our paper](http://pages.cs.wisc.edu/~anhai/papers1/deepmatcher-sigmod18.pdf)** for a more detailed explanation. Here we give briefly describe the intuition behind these four model types:
* **sif:** This model considers the **words** present in each attribute value pair to determine a match or non-match. It does not take word order into account.
* **rnn:** This model considers the **sequences of words** present in each attribute value pair to determine a match or non-match.
* **attention:** This model considers the **alignment of words** present in each attribute value pair to determine a match or non-match. It does not take word order into account.
* **hybrid:** This model considers the **alignment of sequences of words** present in each attribute value pair to determine a match or non-match.

`deepmatcher` is highly customizable and allows you to tune almost every aspect of the neural network model for your application scenario. **[This tutorial](https://github.com/sidharthms/deepmatcher/tree/master/examples/customize_network.ipynb)** discusses the structure of `MatchingModel`s and how they can be customized.

For this tutorial, let's create a `hybrid` model for entity matching:

In [ ]:
model = dm.MatchingModel(attr_summarizer='hybrid')

## Step 3. Train model

Next, we train the defined neural network model using the processed training and validation data. To do so, we call the `run_train` method which takes the following basic parameters:

* **train:** The processed training dataset object (of type `MatchingDataset`).
* **validation:** The processed validation dataset object (of type `MatchingDataset`).
* **epochs:** Number of times to go over the entire `train` data for training the model.
* **batch_size:** Number of labeled examples (tuple pairs) to use for each training step. This value may be increased if you have a lot of training data and would like to speed up training. The optimal value is dataset dependent.
* **best_save_path:** Path to save the best model.
* **pos_weight**: The ratio of the weight of positive examples (matches) to weight of negative examples (non-matches). This value should be increased if you have fewer matches than non-matches in your data. The optimal value is dataset dependent.

Many other aspects of the training algorithm can be customized. For details on this, please refer the API documentation for **[run_train]()**

In [ ]:
model.run_train(
    train,
    validation,
    epochs=15,
    batch_size=16,
    best_save_path='hybrid_model.pth',
    pos_weight=1.3)

## Step 4. Apply model to new data

#### Evaluating on test data
Now that we have a trained model for entity matching, we can now evaluate its accuracy on test data, to estimate the performance of the model on unlabeled data.

In [5]:
# Compute F1 on test set
sif_model.run_eval(test)

===>  EVAL Epoch 6 :


0% [██████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:00


Finished Epoch 6 || Run Time:    0.3 | Load Time:    0.7 || F1:  58.17 | Prec:  61.03 | Rec:  55.56 || Ex/s: 2228.61



58.165548098434

#### Getting predictions on unlabeled data

We finally apply the trained model to unlabeled data to get predictions. To do this, we call the `run_prediction` method which takes a processed data set object and returns a `pandas` dataframe containing tuple pair IDs (`id` column) and the corresponding match / non-match predictions (`prediction` column).

In [ ]:
predictions = model.run_prediction(unlabeled)
predictions.head()

You may optionally set the `output_attributes` parameter to also include all attributes present in the original input table. As mentioned earlier, the processed attribute values will likely look a bit different from the attribute values in the input CSV files due to modifications such as tokenization and lowercasing.

In [ ]:
predictions = model.run_prediction(unlabeled, output_attributes=True)
predictions.head()

You can then save these predictions to CSV and use them for downstream tasks.

In [ ]:
predictions.to_csv('sample_data/unlabeled_predictions.csv')

#### Getting predictions on labeled data

You can also get predictions for labeled data such as validation data. To do so, you can simply call the `run_prediction` method passing the validation data as argument.

In [ ]:
valid_predictions = model.run_prediction(validation, output_processed_attributes=True)
valid_predictions.head()